# Assignment - Game: UNO

## Instructions

This is a **self-contained notebook** - everything you need is here!

### Quick Start
1. **Run all cells** up to Section 4 (this loads the game client)
2. **Implement your solver** in Section 5
3. **Update STUDENT_TOKEN** in Section 6
4. **Play the game** by running Section 6

### What You Need To Do
- Focus ONLY on implementing `my_agent()` function (Section 5)
- You can also use `manual_player_solver` to play manually if you want
- Everything else is provided for you!

### About UNO
UNO is a classic card game:
- Players try to get rid of all their cards first
- Match cards by color or number/symbol
- Special cards: Skip, Reverse, Draw 2, Wild, Wild Draw 4
- Must say "UNO" when you have one card left (automatic)
- Strategic card management and timing is key

---
## Section 1: Setup

**Run this cell (no changes needed)**

In [7]:
import requests
import json
import time
import random
from typing import List, Optional, Tuple, Any, Dict

print("✅ Dependencies imported")

BASE_URL = 'https://ie-aireasoning-gr4r5bl6tq-ew.a.run.app'

print("✅ Configuration loaded")

✅ Dependencies imported
✅ Configuration loaded


---
## Section 2: Game Client Library

**Run this cell (no changes needed)**

In [8]:
class GameClient:
    def __init__(self, base_url: str, token: str, debug: bool = False):
        self.base_url = base_url.rstrip('/')
        self.token = token
        self.debug = debug

    def _make_request(self, endpoint: str, params: dict, max_retries: int = 10) -> dict:
        params['TOKEN'] = self.token
        url = f'{self.base_url}{endpoint}'

        for attempt in range(max_retries):
            try:
                if self.debug:
                    print(f"[DEBUG] Request: {endpoint}")
                    print(f"[DEBUG] Params: {params}")

                response = requests.get(url, params=params, timeout=30)

                if self.debug:
                    print(f"[DEBUG] Response [{response.status_code}]: {response.text[:200]}")

                if response.status_code == 200:
                    if response.text:
                        try:
                            return response.json()
                        except (json.JSONDecodeError, ValueError) as e:
                            if self.debug:
                                print(f"[DEBUG] Non-JSON response: {response.text[:100]}")
                            return {}
                    return {}
                else:
                    print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")

            except requests.exceptions.Timeout:
                print(f"⚠️  Request timeout (attempt {attempt + 1}/{max_retries})")
            except requests.exceptions.RequestException as e:
                print(f"⚠️  Request error: {e} (attempt {attempt + 1}/{max_retries})")
            except Exception as e:
                print(f"⚠️  Unexpected error: {type(e).__name__}: {e} (attempt {attempt + 1}/{max_retries})")

            if attempt < max_retries - 1:
                time.sleep(1)

        raise Exception(f"Failed to connect to {endpoint} after {max_retries} attempts")

    def create_match(self, game_type: str, num_games: int, multiplayer: bool = False) -> str:
        response = self._make_request('/new-match', {
            'game-type': game_type,
            'num-games': str(num_games),
            'multi-player': 'True' if multiplayer else 'False'
        })

        if 'match-id' not in response:
            print(f"❌ Server response missing 'match-id'. Response: {response}")
            raise KeyError(f"Server response missing 'match-id'. Got: {response}")

        return response['match-id']

    def join_match(self, match_id: str) -> dict:
        response = self._make_request('/join-match', {
            'match-id': match_id
        })
        return response

    def get_game_state(self, match_id: str, game_index: int) -> dict:
        return self._make_request('/game-state-in-match', {
            'match-id': match_id,
            'game-index': str(game_index)
        })

    def get_match_state(self, match_id: str) -> dict:
        return self._make_request('/match-state', {
            'match-id': match_id
        })

    def make_move(self, match_id: str, player: str, move: Any) -> bool:
        move_str = move if isinstance(move, str) else json.dumps(move)

        self._make_request('/make-move-in-match', {
            'match-id': match_id,
            'player': player,
            'move': move_str
        })
        return True

print("✅ GameClient loaded")


def play_game(solver, base_url: str, token: str, game_type: str, game_class,
              multiplayer: bool = False, match_id: Optional[str] = None,
              num_games: int = 1, debug: bool = False, verbose: bool = True) -> Tuple:
    client = GameClient(base_url, token, debug=debug)

    if match_id is None:
        if verbose:
            print(f"🎮 Creating new match: {num_games} x {game_type}")
        match_id = client.create_match(game_type, num_games, multiplayer)
        if verbose:
            print(f"   Match ID: {match_id}")

    if verbose:
        print(f"🔗 Joining match {match_id}...")
    match = client.join_match(match_id)
    player = match['player']
    num_games = match.get('num-games', num_games)
    if verbose:
        print(f"   You are player: {player}")

    game_state = client.get_game_state(match_id, 0)
    if game_state['status'] == 'waiting':
        if verbose:
            print("⏳ Waiting for opponent to join...")
        while game_state['status'] == 'waiting':
            time.sleep(2)
            game_state = client.get_game_state(match_id, 0)

    all_results = []
    wins = losses = draws = 0

    while True:
        match_state = client.get_match_state(match_id)
        if match_state['status'] != 'in_progress':
            break
        game_num = match_state['current-game-index']

        if verbose:
            print(f"\n{'='*50}")
            print(f"🎮 GAME {game_num + 1}/{num_games}")
            print(f"{'='*50}\n")

        game_state = client.get_game_state(match_id, game_num)
        if 'my-player' in game_state:
            player = game_state['my-player']

        game = game_class(game_state['state'], game_state['status'], game_state['player'], player)

        while game_state['status'] != 'complete':
            game_state = client.get_game_state(match_id, game_num)
            if 'my-player' in game_state:
                player = game_state['my-player']
            if 'winner' in game_state:
                break

            game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
            if game.is_terminal():
                break

            if verbose:
                game.print_state()

            if game.current_player == player:
                if verbose:
                    print(f"🤔 Your turn (Player {player})...")
                try:
                    move = solver(game)
                    if verbose:
                        print(f"   Move: {move}")
                    client.make_move(match_id, player, move)
                except Exception as e:
                    print(f"❌ Error in solver: {e}")
                    import traceback
                    traceback.print_exc()
                    break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent (Player {game.current_player})...")
                time.sleep(2)

        if verbose:
            game.print_state()
            print("=" * 40)

        winner = game_state.get('winner')
        if winner == '-':
            if verbose:
                print("🤝 DRAW!")
            result = 'draw'
            draws += 1
        elif winner == player:
            if verbose:
                print("🎉 You WON!")
            result = 'win'
            wins += 1
        else:
            if verbose:
                print("😞 You LOST")
            result = 'loss'
            losses += 1

        all_results.append((result, player, winner))

        if verbose and num_games > 1:
            print(f"\n📊 Record: {wins}W - {losses}L - {draws}D")

    return {
        'wins': wins, 'losses': losses, 'draws': draws,
        'total_games': num_games,
        'win_rate': wins / num_games if num_games > 0 else 0,
        'player': player, 'match_id': match_id
    }, all_results

print("✅ play_game loaded")

✅ GameClient loaded
✅ play_game loaded


---
## Section 3: Game State Class

**Run this cell (no changes needed)**

In [9]:
class UnoGame:
    """Represents UNO game state."""

    def __init__(self, state: str, status: str, current_player: str, my_player: str):
        self.state_str = state
        self.status = status
        self.current_player = current_player
        self.my_player = my_player
        self._state = None

    @property
    def state(self) -> Dict:
        if self._state is None:
            self._state = json.loads(self.state_str)
        return self._state

    def get_my_hand(self) -> List[Dict]:
        return self.state['hands'].get(self.my_player, [])

    def get_hand_sizes(self) -> Dict[str, int]:
        hands = self.state.get('hands', {})
        return {player: len(hand) for player, hand in hands.items()}

    def get_current_color(self) -> str:
        return self.state.get('current_color', '')

    def get_top_card(self) -> Dict:
        discard_pile = self.state.get('discard_pile', [])
        return discard_pile[-1] if discard_pile else {}

    def get_discard_pile(self) -> List[Dict]:
        """Get the full discard pile (all cards that have been played)."""
        return self.state.get('discard_pile', [])

    def get_discard_pile_size(self) -> int:
        """Get the number of cards in the discard pile."""
        return len(self.state.get('discard_pile', []))

    def is_terminal(self) -> bool:
        return self.status == 'complete'

    def _can_play_card(self, card: Dict, top_card: Dict, current_color: str) -> bool:
        if not top_card:
            return True
        if card.get('type') == 'wild':
            return True
        if card.get('color') == current_color:
            return True
        if card.get('value') == top_card.get('value'):
            return True
        return False

    def get_valid_moves(self) -> List[Dict]:
        if self.current_player != self.my_player:
            return []

        hand = self.get_my_hand()
        top_card = self.get_top_card()
        current_color = self.get_current_color()
        valid_moves = []

        for i, card in enumerate(hand):
            if self._can_play_card(card, top_card, current_color):
                move = {'type': 'play', 'card_index': i, 'card': card, 'call_uno': len(hand) == 2}
                if card.get('type') == 'wild':
                    for color in ['red', 'blue', 'green', 'yellow']:
                        color_move = move.copy()
                        color_move['color_choice'] = color
                        valid_moves.append(color_move)
                else:
                    valid_moves.append(move)

        valid_moves.append({'type': 'draw', 'count': 1})
        return valid_moves

    def print_state(self):
        print(f"\n{'='*50}")
        print(f"Current Turn: Player {self.current_player}")
        print(f"Current Color: {self.get_current_color().upper()}")
        top_card = self.get_top_card()
        if top_card:
            print(f"Top Card: {top_card.get('color')} {top_card.get('value')}")
        hand_sizes = self.get_hand_sizes()
        print("\nHand Sizes:")
        for p in sorted(hand_sizes.keys()):
            if p != self.my_player:
                print(f"  Player {p}: {hand_sizes[p]} cards")
        my_hand = self.get_my_hand()
        print(f"\nYour Hand ({len(my_hand)} cards):")
        for i, card in enumerate(my_hand):
            print(f"  {i}: {card.get('color')} {card.get('value')}")
        print('='*50)

print("✅ UnoGame class loaded")

✅ UnoGame class loaded


---
## Section 4: Manual solver

**Run this cell (no changes needed)**

In [10]:
def manual_player_solver(game: UnoGame) -> Dict:
    """
    Interactive manual player - YOU choose your moves!
    """
    game.print_state()

    valid_moves = game.get_valid_moves()

    if not valid_moves:
        print("No valid moves!")
        return {'type': 'draw', 'count': 1}

    print(f"\n🎮 YOUR TURN (Player {game.my_player})!")
    print("\nValid moves:")

    move_list = []
    move_idx = 0

    # Display playable cards
    play_moves = [m for m in valid_moves if m.get('type') == 'play']
    if play_moves:
        print("\n🎴 Cards you can play:")
        for move in play_moves:
            card_idx = move.get('card_index')
            card = move.get('card', {})
            color = card.get('color', '?').upper()
            value = card.get('value', '?')

            if card.get('type') == 'wild':
                choice = move.get('color_choice', 'red').upper()
                print(f"  {move_idx}: Play WILD as {choice}")
            else:
                print(f"  {move_idx}: Play {color} {value} (card #{card_idx})")

            move_list.append(move)
            move_idx += 1

    # Display draw option
    print(f"\n  {move_idx}: Draw a card")
    move_list.append({'type': 'draw', 'count': 1})

    # Get player choice
    while True:
        try:
            choice = input(f"\nEnter move number (0-{len(move_list)-1}, or 'q' to quit): ").strip()

            if choice.lower() == 'q':
                raise KeyboardInterrupt()

            idx = int(choice)
            if 0 <= idx < len(move_list):
                selected_move = move_list[idx]

                # If it's a play move with a wild card, ask for color choice
                if selected_move.get('type') == 'play' and selected_move.get('card', {}).get('type') == 'wild':
                    print("\nWild card color choices:")
                    colors = ['red', 'blue', 'green', 'yellow']
                    for i, c in enumerate(colors):
                        print(f"  {i}: {c.upper()}")

                    while True:
                        try:
                            color_choice = input("Choose color (0-3): ").strip()
                            color_idx = int(color_choice)
                            if 0 <= color_idx < len(colors):
                                selected_move['color_choice'] = colors[color_idx]
                                break
                            else:
                                print(f"❌ Invalid choice! Enter 0-{len(colors)-1}")
                        except ValueError:
                            print("❌ Invalid input! Enter a number.")

                return selected_move
            else:
                print(f"❌ Invalid index! Choose 0-{len(move_list)-1}")

        except ValueError:
            print("❌ Invalid input! Enter a number or 'q' to quit.")
        except KeyboardInterrupt:
            print("\n👋 Thanks for playing!")
            raise

print("✅ Manual player solver loaded")

✅ Manual player solver loaded


---
## Section 5: YOUR SOLVER IMPLEMENTATION

**⭐ THIS IS WHERE YOU WRITE YOUR CODE! ⭐**

### Available Methods

```python
game.get_my_hand()                        # List of cards in your hand
game.get_hand_sizes()                     # Dict of player hand sizes
game.get_current_color()                  # Current active color
game.get_top_card()                       # Card on top of discard pile
game.get_discard_pile()                   # Full discard pile history (all played cards)
game.get_discard_pile_size()              # Number of cards in discard pile
game.get_valid_moves()                    # All valid moves you can make
game.is_terminal()                        # Whether game is finished
game.print_state()                        # Print current game state
```

### Move Format
- Play a card: `{'type': 'play', 'card_index': i, 'card': {...}, 'color_choice': 'red'}`
  - `card_index`: Index of card in your hand (0-indexed)
  - `color_choice`: Only required for Wild cards (one of: 'red', 'blue', 'green', 'yellow')
- Draw a card: `{'type': 'draw', 'count': 1}`

### Card Information
Card dict contains: `{'color': 'red', 'value': '5', 'type': 'normal'}`
- Colors: 'red', 'blue', 'green', 'yellow'
- Values: '0'-'9', 'skip', 'reverse', 'draw2'
- Types: 'normal', 'action', 'wild'

### Strategic Tips
- **Discard Pile History**: Use `game.get_discard_pile()` to see all cards that have been played. This allows you to implement card-counting strategies!
- **Example**: Count which high cards are still in the deck vs. already played to make better decisions
- **Hand Tracking**: Use `game.get_hand_sizes()` to track how many cards opponents have

In [11]:
from typing import Dict, Any, Tuple
from collections import Counter

def my_agent(game: UnoGame) -> Dict:
    """
    Heuristic-based UNO AI agent with discard-pile-informed card counting.
    Includes threat-aware action timing, late-game color control, self endgame heuristics,
    point-shedding logic, and improved wild card scoring.
    """

    valid_moves = game.get_valid_moves()
    play_moves = [m for m in valid_moves if m.get('type') == 'play']

    if not play_moves:
        return {'type': 'draw', 'count': 1}

    # Get game state information
    my_hand = game.get_my_hand()
    my_hand_size = len(my_hand)
    hand_sizes = game.get_hand_sizes()
    current_color = game.get_current_color()
    discard_pile = game.get_discard_pile()
    discard_size = len(discard_pile)

    COLORS = ['red', 'blue', 'green', 'yellow']
    ACTION_VALUES = ['skip', 'reverse', 'draw2']

    # Card point values for point-shedding (if scoring matters)
    def card_points(card: Dict) -> int:
        """Estimate point value of a card (for point-shedding heuristic)."""
        card_type = card.get('type', 'normal')
        value = card.get('value', '')
        if card_type == 'wild':
            if value == 'wild_draw4':
                return 50
            return 40
        if card_type == 'action':
            return 20
        if value.isdigit():
            return int(value)
        return 10

    def card_key(card: Dict) -> Tuple[Any, str, str]:
        card_type = card.get('type', 'normal')
        color = card.get('color') if card_type != 'wild' else None
        value = card.get('value', '')
        return (color, value, card_type)

    def build_full_deck_counts() -> Dict[Tuple[Any, str, str], int]:
        counts = {}
        for color in COLORS:
            counts[(color, '0', 'normal')] = 1
            for val in map(str, range(1, 10)):
                counts[(color, val, 'normal')] = 2
            for action in ACTION_VALUES:
                counts[(color, action, 'action')] = 2
        counts[(None, 'wild', 'wild')] = 4
        counts[(None, 'wild_draw4', 'wild')] = 4
        return counts

    def compute_remaining_counts() -> Dict[Tuple[Any, str, str], int]:
        full_counts = build_full_deck_counts()
        seen_cards = discard_pile + my_hand  # Cards we know are not in opponents' hands
        for card in seen_cards:
            key = card_key(card)
            if key in full_counts:
                full_counts[key] = max(0, full_counts[key] - 1)
        return full_counts

    remaining_counts = compute_remaining_counts()
    remaining_color_counts = {color: 0 for color in COLORS}
    remaining_action_counts = {action: 0 for action in ACTION_VALUES}

    for (color, value, card_type), count in remaining_counts.items():
        if card_type != 'wild' and color in remaining_color_counts:
            remaining_color_counts[color] += count
        # Track remaining action cards
        if card_type == 'action' and value in ACTION_VALUES:
            remaining_action_counts[value] += count

    # Count cards by color in our hand
    color_counts = {color: 0 for color in COLORS}
    for card in my_hand:
        if card.get('color') in color_counts:
            color_counts[card.get('color')] += 1

    dominant_color = max(color_counts, key=lambda c: color_counts[c])
    dominant_count = color_counts[dominant_color]
    dominant_ratio = dominant_count / max(1, my_hand_size)
    has_non_wild_play = any(
        move.get('card', {}).get('type') != 'wild' for move in play_moves
    )

    # Find opponent with fewest cards (biggest threat)
    opponents = {p: size for p, size in hand_sizes.items() if p != game.my_player}
    min_opponent_hand = min(opponents.values()) if opponents else 7

    # Self endgame: are we close to winning?
    is_self_endgame = my_hand_size <= 3

    def threat_multiplier() -> float:
        if min_opponent_hand <= 1:
            return 2.5
        if min_opponent_hand <= 2:
            return 1.7
        if min_opponent_hand <= 3:
            return 1.3
        return 1.0

    threat_factor = threat_multiplier()

    def color_scarcity_bonus(card_color: str) -> float:
        remaining = remaining_color_counts.get(card_color, 0)
        base_bonus = max(0, 10 - remaining) * 0.3
        # Scale with deck exhaustion: as discard pile grows, scarcity matters more
        # Early game (0-20 cards): 1.0x, Mid game (20-40): 1.2x, Late game (40+): 1.5x
        exhaustion_factor = 1.0 + (discard_size / 100.0)  # Scales from 1.0 to ~1.5
        return base_bonus * exhaustion_factor

    def action_scarcity_bonus(action_value: str) -> float:
        """Bonus for playing action cards when few remain in deck."""
        remaining = remaining_action_counts.get(action_value, 0)
        # Maximum 8 action cards per type (2 per color × 4 colors)
        # Bonus increases as fewer remain (opponents less likely to have counters)
        if remaining <= 2:
            return 4.0  # Very scarce - high bonus
        elif remaining <= 4:
            return 2.0  # Moderately scarce
        elif remaining <= 6:
            return 1.0  # Somewhat scarce
        return 0.0  # Plenty remain

    def late_game_color_bonus(card_color: str) -> float:
        if discard_size < 20:
            return 0.0
        if card_color == dominant_color:
            return 4.0
        return 0.0

    def score_wild_with_color(wild_card: Dict, test_color: str) -> float:
        """Score a wild card move with a specific color choice."""
        score = 0.0

        # 3a. Current color weakness: if we are weak in current color
        current_color_count = color_counts.get(current_color, 0)
        if current_color_count == 0:
            score += 50.0  # EXTREMELY high bonus if completely stuck - ALWAYS use wild instead of drawing!
        elif current_color_count <= 1:
            score += 15.0  # Moderate bonus if weak in current color

        # 3b. Color control: favor colors we have many of and are scarce
        score += (color_counts.get(test_color, 0) * 3.0)
        score += color_scarcity_bonus(test_color)

        # 3c. Dominant color reinforcement
        if dominant_ratio >= 0.6 and test_color == dominant_color:
            score += 6.0

        # 3d. Threat response: opponent close to winning
        if min_opponent_hand <= 1:
            score += 30.0  # Very aggressive when opponent has 1 card - CRITICAL
        elif min_opponent_hand <= 2:
            score += 20.0  # Increased - be more aggressive

        # 3e. Early game penalty: don't waste wilds early if we have other plays
        if min_opponent_hand > 4 and has_non_wild_play:
            score -= 6.0

        # Self endgame: prefer colors we can finish with
        if is_self_endgame and test_color == dominant_color:
            score += 8.0

        return score

    def score_move(move: Dict) -> float:
        """Score a move based on strategic heuristics and card counting."""
        if move.get('type') != 'play':
            return -1000.0  # Strongly prefer playing over drawing if any play is possible

        card = move.get('card', {})
        card_type = card.get('type', 'normal')
        card_value = card.get('value', '')
        card_color = card.get('color', '')
        chosen_color = move.get('color_choice', current_color)

        score = 0.0

        # 1. HAND MANAGEMENT: Prefer colors we have many of
        if card_type == 'wild':
            score += color_counts.get(chosen_color, 0) * 2.0
        else:
            score += color_counts.get(card_color, 0) * 1.5

        # 2. ACTION CARDS: Threat-aware timing + scarcity bonus
        if card_type == 'action' or card_value in ACTION_VALUES:
            score += (4.0 * threat_factor)
            if min_opponent_hand <= 2:
                score += 18.0  # Increased - be more aggressive blocking
            if min_opponent_hand <= 1:
                score += 35.0  # Much higher bonus when opponent has 1 card - CRITICAL - MUST BLOCK
            # Self endgame: action cards help us finish
            if is_self_endgame:
                score += 5.0
            # Action card scarcity: bonus when few remain (opponents less likely to have counters)
            score += action_scarcity_bonus(card_value)

        # 3. WILD CARDS: Use strategically for color control
        # Note: Wild Draw4 is handled separately in section 7 (emergency only)
        if card_type == 'wild' and card_value != 'wild_draw4':
            # Base bonus for regular wilds - always prefer over drawing
            score += 10.0
            # Regular wild cards get strategic bonuses
            score += score_wild_with_color(card, chosen_color)

        # 4. CARD COUNTING / LATE GAME COLOR CONTROL / DOMINANT COLOR
        if card_type != 'wild' and card_color in COLORS:
            # Scarcity: deny opponents a color that is nearly exhausted
            score += color_scarcity_bonus(card_color)
            # Late game: reinforce dominant color
            score += late_game_color_bonus(card_color)
            # Dominant color reinforcement when we are very skewed
            if dominant_ratio >= 0.6 and card_color == dominant_color:
                score += 3.0

        # 5. NUMBER CARDS: minor preference when we have many of that color
        if card_type == 'normal' and card_value.isdigit():
            score += color_counts.get(card_color, 0) * 0.5

        # 6. MATCHING CURRENT COLOR: small tie-breaker bonus
        if card_color == current_color:
            score += 2.0

        # 7. WILD DRAW 4: Emergency card - save for critical moments ONLY
        if card_value == 'wild_draw4':
            # Only valuable when opponent is truly close to winning
            if min_opponent_hand <= 2:
                # Emergency use - high priority, choose best color
                score += (25.0 * threat_factor)  # Emergency use
                # Minimal color selection for emergency (just pick dominant color)
                if chosen_color == dominant_color:
                    score += 3.0
            elif min_opponent_hand <= 3:
                score += 15.0  # Moderate threat
                if chosen_color == dominant_color:
                    score += 2.0
            else:
                # Early game: VERY strongly discourage using draw4
                score -= 50.0  # Very strong penalty for wasting it early
                if has_non_wild_play:
                    score -= 20.0  # Extra penalty if we have alternatives
                # Only use if we're truly stuck (no matching colors at all)
                if color_counts.get(current_color, 0) == 0:
                    score += 10.0  # Small bonus if we have no cards of current color, but still penalized

        # 8. PRESERVE UNIQUE COLORS: avoid burning our last card of a color
        if card_type != 'wild' and color_counts.get(card_color, 0) == 1:
            score -= 2.0

        # 9. SELF ENDGAME: When we're close to winning (≤3 cards)
        if is_self_endgame:
            # Prefer playing cards of our dominant color (helps finish)
            if card_type != 'wild' and card_color == dominant_color:
                score += 6.0  # Moderate bonus, not overwhelming
            # Prefer finishing with numbers over action cards (more reliable)
            if card_type == 'normal' and card_value.isdigit():
                score += 3.0

        return score

    # Score all moves (get_valid_moves already provides separate moves for each wild color choice)
    scored_moves = [(score_move(move), move) for move in play_moves]
    scored_moves.sort(reverse=True, key=lambda x: x[0])
    best_move = scored_moves[0][1]

    return best_move


def simple_agent(game) -> Dict[str, Any]:
    """
    Simple heuristic UNO agent.

    Strategy:
    - Always play if possible, otherwise draw.
    - Prefer cards of the color we have the most of (keeps hand flexible).
    - Prefer number cards over action / wild cards (shed points).
    - Use action cards more aggressively if an opponent is close to winning.
    - Avoid using Wild Draw 4 if we still have normal playable cards.
    """
    valid_moves = game.get_valid_moves()
    play_moves = [m for m in valid_moves if m.get("type") == "play"]

    # If we can't play anything, draw one card
    if not play_moves:
        return {"type": "draw", "count": 1}

    my_hand = game.get_my_hand()
    hand_sizes = game.get_hand_sizes()
    current_color = game.get_current_color()
    COLORS = ["red", "blue", "green", "yellow"]

    # Count colors in our hand to find dominant color
    color_counts = {c: 0 for c in COLORS}
    for card in my_hand:
        c = card.get("color")
        if c in color_counts:
            color_counts[c] += 1

    dominant_color = max(color_counts, key=color_counts.get) if color_counts else None

    # Find smallest opponent hand size (threat level)
    opponents = [size for p, size in hand_sizes.items() if p != game.my_player]
    min_opp = min(opponents) if opponents else None

    def score(move: Dict[str, Any]) -> float:
        card = move.get("card", {})
        card_type = card.get("type", "normal")
        value = card.get("value", "")
        color = card.get("color") or move.get("color_choice")
        s = 0.0

        # 1) Prefer keeping to our dominant color
        if color and color == dominant_color:
            s += 3.0

        # 2) Small bonus for matching current color (smooth play)
        if color and color == current_color:
            s += 1.0

        # 3) Prefer number cards (shed points first, keep powerful cards)
        if card_type == "normal" and str(value).isdigit():
            s += 2.0

        # 4) Action cards: more valuable if someone is close to going out
        if card_type == "action" and min_opp is not None:
            if min_opp <= 2:
                s += 4.0
            elif min_opp <= 3:
                s += 2.0

        # 5) Wild (non-draw4): generally useful, small bonus
        if card_type == "wild" and value != "wild_draw4":
            s += 3.0

        # 6) Wild Draw 4: avoid wasting it if we have normal plays
        if value == "wild_draw4":
            has_non_wild_play = any(
                pm.get("card", {}).get("type") != "wild" for pm in play_moves
            )
            if has_non_wild_play:
                s -= 5.0  # discourage early use
            else:
                s += 5.0  # good emergency option

        # 7) Tiny tie-breaker: slightly prefer lower card_index (older cards)
        if "card_index" in move:
            s -= move["card_index"] * 0.01

        return s

    # Pick the move with the highest score
    best_move = max(play_moves, key=score)
    return best_move

print("✅ Solver defined - customize my_agent() with your strategy!")
print("✅ Simple agent also available - use simple_agent() for a cleaner implementation!")

✅ Solver defined - customize my_agent() with your strategy!
✅ Simple agent also available - use simple_agent() for a cleaner implementation!


---
## Section 6: Play the Game!

**Update STUDENT_TOKEN below and run to play**

You can choose which solver to use:
- `my_agent` - Your AI implementation (default)
- `manual_player_solver` - Interactive manual play

In [15]:
STUDENT_TOKEN = 'RYAN-MUENKER'
SOLVER = simple_agent  # Change to manual_player_solver to play manually!
MULTIPLAYER = False
MATCH_ID = None
NUM_GAMES = 10

try:
    stats, results = play_game(
        solver=SOLVER,
        base_url=BASE_URL,
        token=STUDENT_TOKEN,
        game_type='uno4',
        game_class=UnoGame,
        multiplayer=MULTIPLAYER,
        num_games=NUM_GAMES,
        match_id=MATCH_ID,
        verbose=True
    )

    print("\n📊 Summary:")
    print(f"   Record: {stats['wins']}W - {stats['losses']}L - {stats['draws']}D")
    print(f"   Win Rate: {stats['win_rate']*100:.1f}%")

except Exception as e:
    print(f"❌ Game error: {e}")
    import traceback
    traceback.print_exc()

🎮 Creating new match: 10 x uno4
   Match ID: 3037
🔗 Joining match 3037...
   You are player: 1

🎮 GAME 1/10


Current Turn: Player 2
Current Color: YELLOW
Top Card: yellow 1

Hand Sizes:
  Player 1: 7 cards
  Player 3: 7 cards
  Player 4: 7 cards

Your Hand (7 cards):
  0: yellow 3
  1: green 5
  2: green 7
  3: blue draw2
  4: yellow 2
  5: red skip
  6: red 5
🤔 Your turn (Player 2)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'yellow', 'value': 3, 'type': 'number'}, 'call_uno': False}

Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow 4

Hand Sizes:
  Player 1: 7 cards
  Player 3: 6 cards
  Player 4: 6 cards

Your Hand (6 cards):
  0: green 5
  1: green 7
  2: blue draw2
  3: yellow 2
  4: red skip
  5: red 5
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 2
Current Color: BLUE
Top Card: wild wild

Hand Sizes:
  Player 1: 6 cards
  Player 3: 6 cards
  Player 4: 6 cards

Your Hand (6 cards):
  0: green 5
  1: green 7
  2: blue draw2
  3: yello